In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from operations import OPS

In [3]:
train = datasets.MNIST('./data', train=True, transform=transforms.ToTensor(), download=True)
valid = datasets.MNIST('./data', train=False)

train_loader = DataLoader(train, batch_size=10)

In [4]:
x, y = next(iter(train_loader))

In [5]:
x.shape

torch.Size([10, 1, 28, 28])

In [10]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=10, shuffle=True)

In [11]:
x, y = next(iter(train_loader))

In [12]:
x.shape

torch.Size([10, 1, 28, 28])

In [57]:
class StemNet(nn.Module):
    """ Network stem

    This will always be the beginning of the network.
    DARTS will only recompose modules after the stem.
    For this reason, we define this separate from the
    other modules in the network.

    Args:
        input_dim: the input dimension for your data

        cell_dim: the intermediate dimension size for
                  the remaining modules of the network.
    """
    def __init__(self, in_channels: int=1, cell_dim: int=100, kernel_size=3):
        super(StemNet, self).__init__()
        self.stem = nn.Conv2d(in_channels, cell_dim, kernel_size)

    def forward(self, x):
        return self.stem(x)


In [58]:
class ConvBlock(nn.Module):
    """ ReLu -> Conv1d -> BatchNorm """

    def __init__(self, c_in, c_out, kernel_size, stride, affine=True):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(c_in, c_out, kernel_size=kernel_size, stride=stride)

    def forward(self, x):
        return self.conv(x)

In [100]:
class DilConv(nn.Module):
    """ ReLU Dilated Convolution """

    def __init__(self, c_in, c_out, kernel_size, 
                 stride, padding, dilation, affine=True):
        super(DilConv, self).__init__()

        self.op = nn.Sequential(
            nn.ReLU(inplace=False),

            nn.Conv2d(
                c_in,
                c_in,
                kernel_size=kernel_size,
                stride=stride,
                padding=padding,
                dilation=dilation,
                groups=c_in,
                bias=False
            ),

            nn.Conv2d(
                c_in,
                c_out,
                kernel_size=1,
                padding=0,
                bias=False
            ),

            nn.BatchNorm2d(c_out, affine=affine),
        )

    def forward(self, x):
        return self.op(x)

In [101]:
stem = StemNet()

In [102]:
OPS = {
    'dil_conv': lambda c, stride, affine: DilConv(c, c, 3, stride, 2, 2, affine=affine),
    'conv' : lambda c, stride, affine: ConvBlock(c, c, 3, stride, affine=affine),
}

In [103]:
conv = OPS['conv'](100, 1, True)

In [104]:
dill = OPS['dil_conv'](100, 1, True)

In [105]:
out = stem(x)

In [106]:
out.shape

torch.Size([10, 100, 26, 26])

In [107]:
conv(out).shape

torch.Size([10, 100, 24, 24])

In [108]:
dill(out).shape

torch.Size([10, 100, 26, 26])

In [8]:
torch.Size([100, 1, 26, 26])

torch.Size([100, 1, 26, 26])

In [38]:
x = torch.randn(torch.Size([100, 1, 26, 26]))
y = torch.randn(torch.Size([100, 1, 24, 24]))

In [39]:
x.shape

torch.Size([100, 1, 26, 26])

In [40]:
y.shape

torch.Size([100, 1, 24, 24])

In [43]:
new = torch.zeros_like(x)

In [45]:
new[:, :, :24, :24] = y

In [51]:
new

tensor([[[[ 0.5583, -0.4697, -1.0664,  ...,  0.7064,  0.0000,  0.0000],
          [ 0.6258,  0.0675,  0.6688,  ...,  0.6203,  0.0000,  0.0000],
          [ 1.1937,  0.2701,  0.2217,  ...,  0.0806,  0.0000,  0.0000],
          ...,
          [-0.0659, -0.1802,  0.3372,  ..., -0.3461,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]],


        [[[-1.2648, -0.5133, -0.4088,  ..., -0.4855,  0.0000,  0.0000],
          [-1.3394,  0.4108,  0.6637,  ...,  1.4993,  0.0000,  0.0000],
          [ 0.7185, -0.2766, -0.4765,  ..., -1.1961,  0.0000,  0.0000],
          ...,
          [-0.1028, -0.8230, -0.9398,  ...,  0.7469,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]],


        [[[ 0.7054,  0.6086, -1.7438,  ...,  0.1894,  0.0000,  0.0000],
          [ 0.8151,  0.100

In [48]:
y.shape

torch.Size([100, 1, 24, 24])

In [59]:
x.shape > y.shape

True

In [22]:
from torch.nn.utils.rnn import pad_sequence

In [55]:
result = F.pad(input=y, pad=(1, 1, 1, 1), mode='constant', value=0)

In [56]:
result.shape

torch.Size([100, 1, 26, 26])

In [60]:
seq = [x, y, x, y]

In [64]:
prev = seq[0]

In [67]:
prev = seq[0]
padded = [prev]
for tensor in seq:
    if tensor.shape < prev.shape:
        tensor_pad = F.pad(
            input=tensor, pad=(1, 1, 1, 1), mode='constant', value=0
        )
        padded.append(tensor_pad)
    else:
        padded.append(tensor)
    prev = tensor

In [68]:
sum(padded).shape

torch.Size([100, 1, 26, 26])